In [2]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [3]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [4]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

9.999999999998899e-05

In [5]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

9e-05

In [6]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsFeature(df)
    df = ATRFeature(df)
    df = KeltnerChannelsFeature(df)
    df = RSIFeature(df)
    df = MACDFeature(df)
    df['pivots_l'] = False
    df['pivots_h'] = False

    df['low_perv'] = df.low.shift(1)
    df['high_perv'] = df.high.shift(1)
    df['low_next'] = df.low.shift(-1)
    df['high_next'] = df.high.shift(-1)
    df['pivots_l'] = (df.low_perv > df.low) & (df.low_next > df.low)
    df['pivots_h'] = (df.high_perv < df.high) & (df.high_next < df.high)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df.pivots_l = df.pivots_l.astype(float)
    df.pivots_h = df.pivots_h.astype(float)
    return df

In [7]:
df_an = featureExtraction(df_an)

In [8]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    break
        
        df_an['SL'][i] = df_an.low[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]
                    
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    break
                    
        df_an['SL'][i] = df_an.high[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    

C:\Users\meysam\AppData\Local\Temp\ipykernel_9516\3582309630.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = BUY
C:\Users\meysam\AppData\Local\Temp\ipykernel_9516\3582309630.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.low[i-1]
C:\Users\meysam\AppData\Local\Temp\ipykernel_9516\3582309630.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close[i] - df_an.

In [87]:

feature_col = [
    'open', 'high', 'low', 'close',
    
    'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c',
    'Feat_EMA_c', 'Feat_KeUp_c', 'Feat_KeLo_c',
    
    'Feat_BB_MA_o', 'Feat_BB_UP_o', 'Feat_BB_LW_o',
    'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 
    
    'Feat_BB_MA_l', 'Feat_BB_UP_l', 'Feat_BB_LW_l',
    'Feat_EMA_l', 'Feat_KeUp_l', 'Feat_KeLo_l',
    
    'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
    'Feat_EMA_h', 'Feat_KeUp_h', 'Feat_KeLo_h',
    
    'Feat_ATR_14',
    'Feat_gains','Feat_wins_rma', 'Feat_losses_rma', 'Feat_RSI_14', 
#     'Feat_MACD', 'Feat_SIGNAL_MACD', 'Feat_HIST',
    
        'full_range',# this is added ...
       'body_lower', 'body_upper', 
        'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
    'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [88]:
X = []
Y = [] 
indeces = []
for i in range(10,len(df_an)):
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])
            feature.append(df_an[fc][i-5])
            feature.append(df_an[fc][i-6])
            feature.append(df_an[fc][i-8])
            feature.append(df_an[fc][i-9])
            feature.append(df_an[fc][i-10])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [89]:
feature_name = []
feature_name.append('pivots_h[i-1]')
feature_name.append('pivots_l[i-1]')
feature_name.append('pivots_h[i-2]')
feature_name.append('pivots_l[i-2]')
feature_name.append('pivots_h[i-3]')
feature_name.append('pivots_l[i-3]')
feature_name.append('pivots_h[i-4]')
feature_name.append('pivots_l[i-4]')
for fc in feature_col:
    feature_name.append(fc+'[i]')
    feature_name.append(fc+'[i-1]')
    feature_name.append(fc+'[i-2]')
    feature_name.append(fc+'[i-3]')
    feature_name.append(fc+'[i-4]')

In [90]:
# for i in range(len(feature_name)):
#     print(feature_name[i]," : ",feature[i])

In [91]:
SPLIT_INDEX = int(0.8*len(X))
X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]

In [72]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [73]:
len(feature)

558

In [116]:
clf = MLPClassifier( solver='adam', alpha=1e-4,learning_rate_init=1e-4, # adam
                    hidden_layer_sizes=(600,600), 
                    random_state=1,
                    max_iter=200,
                    verbose=True,warm_start = True,
                    n_iter_no_change=10,
#                     early_stopping = True,
#                     validation_fraction = 0.2 # default is 0.1
                   )

In [117]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 0.91627486
Iteration 2, loss = 0.83590784
Iteration 3, loss = 0.79455123
Iteration 4, loss = 0.75800343
Iteration 5, loss = 0.73339762
Iteration 6, loss = 0.71663550
Iteration 7, loss = 0.71028209
Iteration 8, loss = 0.70092750
Iteration 9, loss = 0.69346056
Iteration 10, loss = 0.68565273
Iteration 11, loss = 0.68291992
Iteration 12, loss = 0.67589588
Iteration 13, loss = 0.66960023
Iteration 14, loss = 0.66553399
Iteration 15, loss = 0.65610225
Iteration 16, loss = 0.65192378
Iteration 17, loss = 0.64909468
Iteration 18, loss = 0.64183345
Iteration 19, loss = 0.63933543
Iteration 20, loss = 0.63397516
Iteration 21, loss = 0.62611991
Iteration 22, loss = 0.62306901
Iteration 23, loss = 0.61836906
Iteration 24, loss = 0.60988230
Iteration 25, loss = 0.60452975
Iteration 26, loss = 0.60255402
Iteration 27, loss = 0.59787412
Iteration 28, loss = 0.59317703
Iteration 29, loss = 0.58192395
Iteration 30, loss = 0.57507792
Iteration 31, loss = 0.56946215
Iteration 32, los

C:\Users\meysam\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



MLPClassifier(hidden_layer_sizes=(600, 600), learning_rate_init=0.0001,
              random_state=1, verbose=True, warm_start=True)

In [118]:
y_perdict = clf.predict(X_test)

In [119]:
clf.score(X_train, Y_train)

1.0

In [120]:
clf.score(X_test, Y_test)

0.5569693464430306

In [121]:
conf = confusion_matrix(Y_test, y_perdict)
conf

array([[115, 224,  10],
       [158, 705, 182],
       [  5, 187, 143]], dtype=int64)

In [114]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
#         df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_l[idx-1]==1 and y_perdict[i] ==BUY:
            df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_h[idx-1]==1 and y_perdict[i] ==SELL:
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)

C:\Users\meysam\AppData\Local\Temp\ipykernel_9516\3314508941.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_9516\3314508941.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [115]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

prepare_data...
run_test...
Result:
win rate:  0.3583208395802099
-1.0    428
 2.0    239
Name: result, dtype: int64


In [110]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio =1# abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = 2#abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_9516\4279289003.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [111]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])

min balance  79.20455628882712
